In [56]:
import pandas as pd

In [92]:
# Load the data
income_df = pd.read_csv('C:/Users/abc/Desktop/Projects/corporate-fin-analysis/data_raw/income_stmt_aapl.csv')
balance_df = pd.read_csv('C:/Users/abc/Desktop/Projects/corporate-fin-analysis/data_raw/balance_sheet_aapl.csv')

In [ ]:
# Standardize column names (e.g., "Cost Of Revenue" → "cost_of_revenue")
income_df.columns = income_df.columns.str.lower().str.replace(" ", "_")
balance_df.columns = balance_df.columns.str.lower().str.replace(" ", "_")


# Drop "unnamed:_0" column (artifact of index reset)
balance_df.drop(columns=["unnamed:_0"], inplace=True, errors="ignore")
income_df.drop(columns=["unnamed:_0"], inplace=True, errors="ignore")

In [94]:
# Forward-fill missing quarterly data (e.g., R&D expenses)
income_df.bfill(inplace=True)

In [95]:
# Check for negative equity (illogical for Apple)
if (balance_df['stockholders_equity'] < 0).any():
    raise ValueError("Negative equity detected!")

In [96]:
# Ensure dates are unique
if income_df['date'].duplicated().any():
    income_df = income_df.drop_duplicates(subset=['date'], keep='last')

balance_df['treasury_shares_number'] = balance_df['treasury_shares_number'].fillna(0)



In [97]:
import urllib
from sqlalchemy import create_engine

conn_str = (
    r"DRIVER={ODBC Driver 18 for SQL Server};"
    r"SERVER=DESKTOP-0EA0GRI\SQLEXPRESS;"
    r"DATABASE=corporate-fin-analysis;"
    r"Trusted_Connection=yes;"
    r"Encrypt=no;"
)

conn_uri = f"mssql+pyodbc:///?odbc_connect={urllib.parse.quote_plus(conn_str)}"
engine = create_engine(conn_uri)



In [98]:
#  Load your DataFrames
income_df.to_sql(
    name='income_statement',
    con=engine,
    if_exists='replace',
    index=False
)

balance_df.to_sql(
    name='balance_sheet',
    con=engine,
    if_exists='replace',
    index=False
)

print(" Data loaded successfully!")


 Data loaded successfully!
